In [2]:
import pandas as pd

df = pd.read_csv("data.csv")
df = pd.read_csv("data1.csv")
df = pd.read_csv("data2.csv")
df.head()


,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [5]:
df = df[df['Ratings'] != 3]
df['sentiment'] = df['Ratings'].apply(lambda x: 1 if x >= 4 else 0)


In [7]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

df['clean_review'] = df['Review text'].astype(str).apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df['clean_review'] = df['clean_review'].apply(lemmatize_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=5000)
X = bow.fit_transform(df['clean_review'])
y = df['sentiment']


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_review'])


In [11]:
from gensim.models import Word2Vec

sentences = [review.split() for review in df['clean_review']]
w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=2)


In [12]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


100%|██████████| 440473133/440473133 [05:00<00:00, 1467948.06B/s]


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [14]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [15]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200)

In [16]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train, y_train)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [17]:
from sklearn.metrics import f1_score

y_pred = svm.predict(X_test)
print("F1 Score:", f1_score(y_test, y_pred))

F1 Score: 0.9625899280575541


In [18]:
import joblib

joblib.dump(svm, "sentiment_model.pkl")
joblib.dump(tfidf, "vectorizer.pkl")


['vectorizer.pkl']

In [19]:
negative_reviews = df[df['sentiment'] == 0]

from collections import Counter

words = " ".join(negative_reviews['clean_review']).split()
Counter(words).most_common(20)

[('read', 328),
 ('shuttle', 318),
 ('quality', 292),
 ('product', 247),
 ('bad', 193),
 ('good', 132),
 ('worst', 106),
 ('poor', 97),
 ('qualityread', 94),
 ('one', 80),
 ('goodread', 77),
 ('day', 73),
 ('buy', 67),
 ('productread', 63),
 ('box', 59),
 ('cork', 54),
 ('mavis', 52),
 ('time', 51),
 ('flipkart', 50),
 ('got', 47)]